In [ ]:
from SEQLinkage.Main import *
from SEQLinkage.Utils import *
from SEQLinkage.Runner import *
from SEQLinkage.Core import *

In [ ]:
import os
import concurrent
import time
from itertools import repeat

args = Args().parser.parse_args('--fam ../data/mwe_normal_fam.csv --vcf ../data/first1000snp_full_samples.vcf.gz --anno ../data/first1000_chr1_multianno.csv --pop ../data/full_sample_fam_pop.txt -f MERLIN --build hg38 --freq AF -o ../data/first1000pickle --bin 1 --maf-cutoff 0.05'.split())

args = Args().parser.parse_args('--fam ../data/new_trim_ped_famless17_no:xx.fam --vcf /mnt/mfs/statgen/alzheimers-family/linkage_files/geno/full_sample/vcf/full_sample.vcf.gz --anno MWE/annotation/EFIGA_NIALOAD_chr19.hg38.hg38_multianno.csv --pop data/full_sample_fam_pop.txt -f MERLIN MEGA2 PLINK LINKAGE --build hg38 --freq AF -o data/fullchr22data -K 0.001 --moi AD -W 0 -M 1 -j 8

In [ ]:
args

Namespace(bin=1.0, blueprint=None, single_markers=False, tfam='../data/mwe_normal_fam.csv', vcf='../data/first1000snp_full_samples.vcf.gz', anno='../data/first1000_chr1_multianno.csv', pop='../data/full_sample_fam_pop.txt', build='hg38', prephased=False, freq='AF', freq_by_fam=None, mle=False, rvhaplo=False, recomb_max=1, recomb_cross_fam=False, rsq=0.0, include_vars=None, maf_cutoff=0.05, chr_prefix=None, output='../data/first1000pickle', format=['MERLIN'], prevalence=None, inherit_mode=None, wild_pen=None, muta_pen=None, theta_max=0.5, theta_inc=0.05, run_linkage=False, output_limit=10, jobs=8, tempdir=None, vanilla=True, quiet=False, debug=False, no_save=False)

In [ ]:
checkParams(args)

MESSAGE: Binary trait detected in [/mnt/mfs/statgen/yin/Github/linkage/SEQpy3/data/mwe_normal_fam.csv]


True

In [ ]:
args.freq 

'AF'

In [ ]:
if args.no_save:
    cache = NoCache()
else:
    cache = Cache(env.cache_dir, env.output, vars(args))
cache.setID('vcf')

In [ ]:
# STEP 1: write encoded data to TPED format
if not args.vanilla and cache.check():
    env.log('Loading regional marker data from archive ...')
    cache.load(target_dir = env.tmp_dir, names = ['CACHE'])
    env.success_counter.value = sum(map(fileLinesCount, glob.glob('{}/*.tped'.format(env.tmp_cache))))
    env.batch = 10
else:
    # load VCF file header
    data = RData(args.vcf, args.tfam,args.anno,args.pop,allele_freq_info=args.freq)
    vs = data.vs
    samples_vcf = data.samples_vcf

if len(samples_vcf) == 0:
    env.error("Fail to extract samples from [{}]".format(args.vcf), exit = True)
env.log('{:,d} samples found in [{}]'.format(len(samples_vcf), args.vcf))
samples_not_vcf = data.samples_not_vcf

anno (1000, 6)


MESSAGE: 7 samples found in FAM file but not in VCF file:
22_1_10, 1036_2, 28_9_100, 28_9_186, 28_9_101, 1036_1, 22_1_20
MESSAGE: 3,461 samples in VCF file will be ignored due to absence in FAM file
MESSAGE: 3,479 samples found in [/mnt/mfs/statgen/yin/Github/linkage/SEQpy3/data/first1000snp_full_samples.vcf.gz]


In [ ]:
len(samples_not_vcf),len(samples_vcf)

(7, 3479)

In [ ]:
if len(data.families) == 0:
    env.error('No valid family to process. ' \
              'Families have to be at least trio with at least one member in VCF file.', exit = True)
if len(data.samples) == 0:
    env.error('No valid sample to process. ' \
              'Samples have to be in families, and present in both TFAM and VCF files.', exit = True)
rewriteFamfile(os.path.join(env.tmp_cache, '{}.tfam'.format(env.output)),
               data.tfam.samples, list(data.samples.keys()) + samples_not_vcf)

In [ ]:
if args.single_markers:
    regions=[]
    for x in vs.GetGenomeCoordinates():
        region_info = (x[0], x[1], x[1], "{}:{}".format(x[0], x[1]), '.', '.', '.')
        if region_info not in regions:
            regions.append(region_info)
    args.blueprint = None
else:
    # load blueprint
    try:
        with open(args.blueprint, 'r') as f:
            regions = [x.strip().split() for x in f.readlines()]
    except IOError:
        env.error("Cannot load regional marker blueprint [{}]. ".format(args.blueprint), exit = True)
env.log('{:,d} families with a total of {:,d} samples will be scanned for {:,d} pre-defined units'.\
        format(len(data.families), len(data.samples), len(regions)))
env.jobs = max(min(args.jobs, len(regions)), 1)

MESSAGE: 3 families with a total of 18 samples will be scanned for 28,325 pre-defined units


## CHR19 APOE

## No parallel

In [ ]:
results1 = {}

In [ ]:
extractor =RegionExtractor(args.vcf, build=env.build,chr_prefix = args.chr_prefix)
maker =            MarkerMaker(args.bin, maf_cutoff = args.maf_cutoff,recomb=False)
writer =             LinkageWriter(len(samples_not_vcf))
for region in regions:
    extractor.getRegion(region)
    maker.getRegion(region)
    writer.getRegion(region)
    isSuccess = True
    for m in [extractor, maker, writer]:
        status = m.apply(data)
        if status == -1:
            with env.chperror_counter.get_lock():
                # previous module failed
                env.chperror_counter.value += 1
        if status == 1:
            with env.null_counter.get_lock():
                env.null_counter.value += 1
        if status == 2:
            with env.trivial_counter.get_lock():
                env.trivial_counter.value += 1
        if status != 0:
            isSuccess = False
            break
    if isSuccess:
        with env.success_counter.get_lock():
            env.success_counter.value += 1
        results1[region[3]]=maker.dtest[region[3]]

In [ ]:
len(results1)

4

## Testing parallel

In [ ]:
extractor =RegionExtractor(args.vcf, build=env.build,chr_prefix = args.chr_prefix)
maker =            MarkerMaker(args.bin, maf_cutoff = args.maf_cutoff,recomb=False)
writer =             LinkageWriter(len(samples_not_vcf))
def run_parallel(region):
    print(region)
    extractor.getRegion(region)
    maker.getRegion(region)
    writer.getRegion(region)
    isSuccess = True
    for m in [extractor, maker, writer]:
        print('in loop',region)
        status = m.apply(data)
        if status == -1:
            with env.chperror_counter.get_lock():
                # previous module failed
                env.chperror_counter.value += 1
        if status == 1:
            with env.null_counter.get_lock():
                env.null_counter.value += 1
        if status == 2:
            with env.trivial_counter.get_lock():
                env.trivial_counter.value += 1
        if status != 0:
            isSuccess = False
            break
    if isSuccess:
        with env.success_counter.get_lock():
            env.success_counter.value += 1
        return region[3],maker.dtest[region[3]]
    return region[3],{}
results1 = {}
start = time.perf_counter()
with concurrent.futures.ProcessPoolExecutor() as executor:
    results = executor.map(run_parallel,regions[2:10])
    for i,j in results:
        results1[i]=j
print(time.perf_counter()-start)

['1', '14361', '29370', 'WASH7P', '1.5299877409602128e-06', '1.94345806118021e-06', '1.136044574393209e-06']
in loop ['1', '14361', '29370', 'WASH7P', '1.5299877409602128e-06', '1.94345806118021e-06', '1.136044574393209e-06']['1', '17368', '17436', 'MIR6859-1@1,MIR6859-2@1,MIR6859-3@1,MIR6859-4@1', '1.217692507120495e-06', '1.5467668502473368e-06', '9.041595098829462e-07']

in loop ['1', '30365', '30503', 'MIR1302-10@1,MIR1302-11@1,MIR1302-2@1,MIR1302-9@1', '2.1295973889038703e-06', '2.705108741548526e-06', '1.5812659765416382e-06']['1', '17368', '17436', 'MIR6859-1@1,MIR6859-2@1,MIR6859-3@1,MIR6859-4@1', '1.217692507120495e-06', '1.5467668502473368e-06', '9.041595098829462e-07']

in loop ['1', '30365', '30503', 'MIR1302-10@1,MIR1302-11@1,MIR1302-2@1,MIR1302-9@1', '2.1295973889038703e-06', '2.705108741548526e-06', '1.5812659765416382e-06']['1', '34610', '36081', 'FAM138A@1,FAM138C@1,FAM138F@1', '2.4732411024120156e-06', '3.1416201771056266e-06', '1.8364278747737466e-06']

in loop ['1',

ValueError: Genotype and sample mismatch for region WASH7P: 2,895 vs 3,479

In [ ]:
regions[:4]

[['1',
  '11868',
  '14362',
  'LOC102725121@1',
  '9.177127474362311e-07',
  '1.1657192989882668e-06',
  '6.814189157634088e-07'],
 ['1',
  '11873',
  '14409',
  'DDX11L1',
  '9.195320788455595e-07',
  '1.1680302941673515e-06',
  '6.82769803434766e-07'],
 ['1',
  '14361',
  '29370',
  'WASH7P',
  '1.5299877409602128e-06',
  '1.94345806118021e-06',
  '1.136044574393209e-06'],
 ['1',
  '17368',
  '17436',
  'MIR6859-1@1,MIR6859-2@1,MIR6859-3@1,MIR6859-4@1',
  '1.217692507120495e-06',
  '1.5467668502473368e-06',
  '9.041595098829462e-07']]

In [ ]:
extractor =RegionExtractor(args.vcf, build=env.build,chr_prefix = args.chr_prefix)
maker =            MarkerMaker(args.bin, maf_cutoff = args.maf_cutoff,recomb=False)
writer =             LinkageWriter(len(samples_not_vcf))

In [ ]:
region = regions[4]

In [ ]:
extractor.getRegion(region)
maker.getRegion(region)
writer.getRegion(region)

In [ ]:
extractor.apply(data)

1

In [ ]:
data

{'1036_3': [],
 '1036_2': [],
 '1036_1': [],
 '1036_5': [],
 '1036_99': [],
 '1036_6': [],
 '1036_4': [],
 '22_1_20': [],
 '22_1_10': [],
 '22_1_99': [],
 '22_1_5': [],
 '22_1_4': [],
 '22_1_2': [],
 '22_1_3': [],
 '28_9_100': [],
 '28_9_103': [],
 '28_9_110': [],
 '28_9_114': [],
 '28_9_105': [],
 '28_9_111': [],
 '28_9_109': [],
 '28_9_108': [],
 '28_9_106': [],
 '28_9_101': [],
 '28_9_186': []}

In [ ]:
def run_parallel(region):
    extractor.getRegion(region)
    maker.getRegion(region)
    writer.getRegion(region)
    isSuccess = True
    for m in [extractor, maker, writer]:
        status = m.apply(data)
        if status == -1:
            with env.chperror_counter.get_lock():
                # previous module failed
                env.chperror_counter.value += 1
        if status == 1:
            with env.null_counter.get_lock():
                env.null_counter.value += 1
        if status == 2:
            with env.trivial_counter.get_lock():
                env.trivial_counter.value += 1
        if status != 0:
            isSuccess = False
            break
    if isSuccess:
        with env.success_counter.get_lock():
            env.success_counter.value += 1
        return region[3],maker.dtest[region[3]]
    return region[3],{}

In [ ]:
run_parallel(regions[6])

('OR4F5', {})

In [ ]:
regions[:3]

[['1',
  '11868',
  '14362',
  'LOC102725121@1',
  '9.177127474362311e-07',
  '1.1657192989882668e-06',
  '6.814189157634088e-07'],
 ['1',
  '11873',
  '14409',
  'DDX11L1',
  '9.195320788455595e-07',
  '1.1680302941673515e-06',
  '6.82769803434766e-07'],
 ['1',
  '14361',
  '29370',
  'WASH7P',
  '1.5299877409602128e-06',
  '1.94345806118021e-06',
  '1.136044574393209e-06']]

In [ ]:
results1

{'LOC102725121@1': {'predata': {'1036': [array(['V6-13273', 'V10-13302', 'V15-13417', 'V20-13687'], dtype='<U9'),
    array([0.1886, 0.0037, 0.1477, 0.0031]),
    array([['1036', '1036_1', '1:', '1:', '1:', '1:'],
           ['1036', '1036_1', '?:', '?:', '?:', '?:'],
           ['1036', '1036_2', '2:', '1:', '2:', '1:'],
           ['1036', '1036_2', '?:', '?:', '?:', '?:'],
           ['1036', '1036_6', '1:', '2:', '1:', '1:'],
           ['1036', '1036_6', '1:', '1:', '1:', '2:'],
           ['1036', '1036_99', '2:', '1|', '2|', '1|'],
           ['1036', '1036_99', '1:', '1|', '1|', '1|'],
           ['1036', '1036_5', '2:', '1|', '2|', '1|'],
           ['1036', '1036_5', '1:', '2|', '1|', '1|'],
           ['1036', '1036_4', '1:', '1|', '1|', '1|'],
           ['1036', '1036_4', '1:', '2|', '1|', '1|'],
           ['1036', '1036_3', '1:', '1|', '1|', '1\\'],
           ['1036', '1036_3', '1:', '1|', '1|', '1\\']], dtype='<U7')],
   '22_1': [array(['V6-13273', 'V13-13380', 'V15-13

In [ ]:
import pickle
with open('../data/dtestpy3_fixedcoder_seqtest_20220126.pickle', 'wb') as handle:
    pickle.dump(a1, handle, protocol=pickle.HIGHEST_PROTOCOL)